In [58]:
import pandas as pd
import numpy as np
import joblib as jl
from tqdm import tqdm
import multiprocessing as mp

In [59]:
dd = jl.load(open('../tmp/auxfile_dd_.pkl', 'rb'))
dd.head()

,ITM_CD,PTNT_ID,SPPLY_DT
6,03415R,233260879,2010-12-29
99,01801T,9856006811,2010-11-11
139,02430X,112017407,2010-01-09
170,09302N,145495445,2010-10-17
171,09302N,9272712359,2010-07-23


In [60]:
metonly = set(pd.read_csv('../mbspbs10pc/data/metformin_items.csv', header=0)['ITM_CD'].values.tolist())
metx = set(pd.read_csv('../mbspbs10pc/data/metformin+x_items.csv', header=0)['ITM_CD'].values.tolist())

In [84]:
# METONLY
grouped = dd.groupby(by='PTNT_ID')
filtered = grouped.filter(lambda x: set(x['ITM_CD']).issubset(metonly)).groupby(by='PTNT_ID')
idx = []
start_date = []
for name, group in filtered:
    idx.append(name)
    start_date.append(group['SPPLY_DT'].min().strftime('%Y-%m-%d'))
print(len(idx))

21637


In [203]:
# MET+X
min_metformin = 10  # metformin only

def condition(group):
    if len(group) <= min_metformin:
        return False
    else:
        sorted_group = group.sort_values(by='SPPLY_DT')
        head = set(sorted_group.head(min_metformin)['ITM_CD'].values) # these must all be metformin
        tail = set(sorted_group.tail(-min_metformin)['ITM_CD'].values) # there should be both in here

        cond1 = head.issubset(metonly)
        cond2 = len(tail.intersection(metonly)) > 0 and len(tail.intersection(metonly)) < len(tail)
        cond3 = len(tail.intersection(metx)) > 0 # handling special case of met+x items introduced after 2014

        return cond1 and (cond2 or cond3)
    

grouped = dd.groupby(by='PTNT_ID')
filtered = grouped.filter(condition).groupby(by='PTNT_ID')

# Init return items
idx, start_date, end_date = list(), list(), list()

# Build output variables
for name, group in tqdm(filtered, desc='Finalizing', leave=False):
    idx.append(name)
    
    sorted_group = group.sort_values(by='SPPLY_DT')
    start_date.append(sorted_group.head(1)['SPPLY_DT'].max().strftime('%Y-%m-%d')) # first date FIXME: max() added for consistency
    filtered_group = sorted_group[~sorted_group['ITM_CD'].isin(metonly)] # get the non metformins
    end_date.append(filtered_group['SPPLY_DT'].min().strftime('%Y-%m-%d'))

In [214]:
# MET2X
def condition(group):
    if len(group) <= min_metformin:
        return False
    else:
        sorted_group = group.sort_values(by='SPPLY_DT')
        head = set(sorted_group.head(min_metformin)['ITM_CD'].values) # these must all be metformin
        tail = set(sorted_group.tail(-min_metformin)['ITM_CD'].values) # there should be NO metformin in here

        cond1 = head.issubset(metonly)
        cond2 = len(tail.intersection(metonly)) == 0
        cond3 = len(tail.intersection(metx)) == 0

        return cond1 and cond2 and cond3
    

grouped = dd.groupby(by='PTNT_ID')
filtered = grouped.filter(condition).groupby(by='PTNT_ID')

In [215]:
# Init return items
idx, start_date, end_date = list(), list(), list()

# Build output variables
for name, group in tqdm(filtered, desc='Finalizing', leave=False):
    idx.append(name)
    
    sorted_group = group.sort_values(by='SPPLY_DT')
    start_date.append(sorted_group.head(1)['SPPLY_DT'].max().strftime('%Y-%m-%d')) # first date FIXME: max() added for consistency
    filtered_group = sorted_group[~sorted_group['ITM_CD'].isin(metonly)] # get the non metformins
    end_date.append(filtered_group['SPPLY_DT'].min().strftime('%Y-%m-%d'))

In [218]:
labels = pd.read_csv('../tmp/labels.csv', header=0, index_col=0)
labels.shape[0] - labels.dropna().shape[0]

38464

In [243]:
ids = dd['PTNT_ID']
idx012 = labels.dropna().index
idx_other = set(ids) - set(idx012)

In [244]:
grouped = dd.groupby(by='PTNT_ID')
filtered = grouped.filter(lambda x: x['PTNT_ID'].values[0] in idx_other).groupby(by='PTNT_ID')

In [245]:
start_date, end_date = list(), list()

# Build output variables
for name, group in tqdm(filtered, desc='Finalizing', leave=False):    
    start_date.append(group['SPPLY_DT'].min().strftime('%Y-%m-%d'))
    end_date.append(group['SPPLY_DT'].max().strftime('%Y-%m-%d'))